# Install `pinn4bhoc` module on Google Colab
  * `USER`: GitHub username
  * `REPO`: GitHub repository
  * `FILES`: Files to be cloned

In [2]:
try:
    import google.colab

    USER  = 'soot-bit'
    REPO  = 'pinn4bhoc'
    FILES= ' '.join([
        'pyproject.toml',
        f'{REPO}/__init__.py', 
        f'{REPO}/nn.py',
        f'{REPO}/orbits', 
        f'{REPO}/utils'])
    
    URL = f'https://github.com/{USER}/{REPO}.git'
    DEVNULL = '> /dev/null 2>&1'
    
    # uninstall repo if installed
    print(f'\t1. uninstall {REPO}')
    !cd /content
    !rm -rf {REPO}  {DEVNULL}
    !pip uninstall -y {REPO} {DEVNULL}
    
    # sparse clone repo
    print(f'\t2. sparse clone {REPO}')
    !git clone --depth 1 --filter=blob:none --sparse {URL} {DEVNULL}
    !cd {REPO}
    !git sparse-checkout init --cone {DEVNULL}
    !git sparse-checkout set {FILES} {DEVNULL}
    !git checkout main {DEVNULL}
    
    # install repo
    print(f'\t3. install {REPO}')
    !pip install . {DEVNULL}
    !cd /content {DEVNULL}

    try:
        exec(f'import {REPO}.nn')
        sys.exit(f'''
    ✅  import {REPO}.nn succeeded!
        ''')
    except ImportError:
        sys.exit(f'''
    ❌ import {REPO}.nn failed
        ''')
except ImportError:
    pass